<a href="https://colab.research.google.com/github/porygon-tech/ECO/blob/main/formulaCheck_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>